# TASK 02


In [9]:
import time
import random
from itertools import product
from collections import defaultdict, deque
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox

In [11]:
SIZE = 9  # Size of the Sudoku grid
BOX_SIZE = 3 #Size of Sudoku box

class SudokuCSP:
    def __init__(self, puzzle):
        self.variables = [(i, j) for i in range(SIZE) for j in range(SIZE)]
        self.values = puzzle
        self.domains = self.initialize_domains()
        self.neighbors = self.initialize_neighbors()

    def initialize_domains(self):
        domains = {}
        for i, j in self.variables:
            if self.values[i][j] == 0:
                domains[(i, j)] = set(range(1, 10))
            else:
                domains[(i, j)] = {self.values[i][j]}
        return domains

    def initialize_neighbors(self):
        neighbors = defaultdict(set)
        for i, j in self.variables:
            for x in range(SIZE):
                neighbors[(i, j)].add((i, x))
                neighbors[(i, j)].add((x, j))
            start_row, start_col = i - i % BOX_SIZE, j - j % BOX_SIZE
            for x, y in product(range(BOX_SIZE), repeat=2):
                neighbors[(i, j)].add((x + start_row, y + start_col))
            neighbors[(i, j)].remove((i, j))
        return neighbors

    def arcs(self):
        return [(var, neighbor) for var in self.variables for neighbor in self.neighbors[var]]

    def constraints(self, Xi, x, Xj, y):
        return x != y

    def revise(self, Xi, Xj):
        revised = False
        for x in list(self.domains[Xi]):
            if not any(self.constraints(Xi, x, Xj, y) for y in self.domains[Xj]):
                self.domains[Xi].remove(x)
                revised = True
        return revised

    def ac3(self):
        queue = deque(self.arcs())
        while queue:
            Xi, Xj = queue.popleft()
            if self.revise(Xi, Xj):
                if not self.domains[Xi]:
                    return False
                for Xk in self.neighbors[Xi]:
                    if Xk != Xj:
                        queue.append((Xk, Xi))
        return True

    def is_complete(self):
        return all(len(self.domains[var]) == 1 for var in self.variables)

    def solution(self):
        if not self.ac3():
            print("AC-3 algorithm failed to find a solution.")
            return False
        solved_puzzle = [[self.domains[(i, j)].pop() for j in range(SIZE)] for i in range(SIZE)]
        return solved_puzzle


class SudokuSolver:
    def __init__(self, puzzle, algo):
        self.algorithm = algo
        self.board = [[0] * SIZE for _ in range(SIZE)]
        if puzzle:
            self.set_puzzle(puzzle)

    def set_puzzle(self, puzzle):
        for i, row in enumerate(puzzle):
            for j, val in enumerate(row):
                self.board[i][j] = val

    def is_valid_move(self, row, col, num):
        for i in range(SIZE):
            if self.board[row][i] == num or self.board[i][col] == num:
                return False
        start_row, start_col = row - row % BOX_SIZE, col - col % BOX_SIZE
        for i, j in product(range(BOX_SIZE), repeat=2):
            if self.board[i + start_row][j + start_col] == num:
                return False
        return True

    def find_empty(self):
        for i in range(SIZE):
            for j in range(SIZE):
                if self.board[i][j] == 0:
                    return i, j
        return None

    def solve_backtracking(self):
        empty = self.find_empty()
        if not empty:
            return True
        row, col = empty
        for num in range(1, SIZE + 1):
            if self.is_valid_move(row, col, num):
                self.board[row][col] = num
                if self.solve_backtracking():
                    return True
                self.board[row][col] = 0
        return False

    def solve_ac3(self):
        csp = SudokuCSP(self.board)
        result = csp.solution()
        if result:
            self.set_puzzle(csp.values)
        return result
    
    def solve_with_algorithm(self):
        if self.algorithm == 'arc':
            solved_puzzle = self.solve_ac3()
        elif self.algorithm == 'backtracking':
            print(self.solve_backtracking())
            solved_puzzle = self.board
        print(solved_puzzle)  # Assign the solved puzzle to solved_puzzle
        return solved_puzzle  # Return the solved puzzle


class SudokuGUI:
    def __init__(self, root, easy_puzzles, medium_puzzles, hard_puzzles):
        self.root = root
        self.EASY_PUZZLES = easy_puzzles
        self.MEDIUM_PUZZLES = medium_puzzles
        self.HARD_PUZZLES = hard_puzzles
        self.puzzle = [[]]
        self.root.title("Welcome to Sudoko Solver!!!")

        # Create frame for Sudoku grid
        grid_frame = ttk.Frame(self.root)
        grid_frame.grid(row=0, column=0, padx=10, pady=10)

        self.board = [[tk.StringVar() for _ in range(SIZE)] for _ in range(SIZE)]
        self.labels = []
        for i in range(SIZE):
            for j in range(SIZE):
                label = tk.Label(grid_frame, width=3, highlightthickness=1, highlightbackground='#000000', textvariable=self.board[i][j])    
                label.grid(row=i, column=j, ipadx=5, ipady=5)
                label.bind('<KeyPress>', lambda event, row=i, col=j: self.validate_input(event, row, col))  # Bind event to validate input
                self.labels.append(label)

        # Create a frame for buttons and dropdown menu
        control_frame = ttk.Frame(self.root)
        control_frame.grid(row=0, column=1, padx=10, pady=10)

        # Create dropdown menu for selecting difficulty level
        difficulty_label = ttk.Label(control_frame, text="Difficulty Level:", font=('TkDefaultFont', 10, 'bold'))
        difficulty_label.grid(row=0, column=0, columnspan=2, padx=10, pady=5)
        self.difficulty_var = tk.StringVar()
        difficulty_combobox = ttk.Combobox(control_frame, textvariable=self.difficulty_var, values=["Easy", "Medium", "Hard"], state="readonly", width=10)
        difficulty_combobox.grid(row=1, column=0, columnspan=2, padx=10, pady=5)
        difficulty_combobox.current(0)  # Set default selection to "Easy"
        difficulty_combobox.bind("<<ComboboxSelected>>", self.load_selected_puzzle)


        # Create Solve button
        solve_button = tk.Button(control_frame, text="Solve", command=self.solve, width=10)
        solve_button.grid(row=2, column=0, padx=10, pady=5)

        # Create Reset button
        reset_button = tk.Button(control_frame, text="Reset", command=self.reset, width=10)
        reset_button.grid(row=2, column=1, padx=10, pady=5)

        # Create radio buttons for choosing puzzle
        puzzle_label = ttk.Label(control_frame, text="Choose Puzzle:", font=('TkDefaultFont', 10, 'bold'))
        puzzle_label.grid(row=3, column=0, columnspan=2, padx=10, pady=5)
        self.puzzle_var = tk.IntVar()
        for i in range(1, 5):
            puzzle_radio = ttk.Radiobutton(control_frame, text=f"Puzzle {i}", variable=self.puzzle_var, value=i, command=self.load_selected_puzzle)
            puzzle_radio.grid(row=4, column=i-1, padx=5, pady=5)

        self.puzzle_var.set(1)

        # Create radio buttons for selecting algorithm
        algorithm_label = ttk.Label(control_frame, text="Choose Algorithm:", font=('TkDefaultFont', 10, 'bold'))
        algorithm_label.grid(row=5, column=0, columnspan=2, padx=10, pady=5)
        self.algorithm_var = tk.StringVar()
        backtracking_radio = ttk.Radiobutton(control_frame, text="Backtracking", variable=self.algorithm_var, value="backtracking")
        backtracking_radio.grid(row=6, column=0, padx=5, pady=5)

        ac3_radio = ttk.Radiobutton(control_frame, text="AC-3", variable=self.algorithm_var, value="arc")
        ac3_radio.grid(row=6, column=1, padx=5, pady=5)
        self.algorithm_var.set("backtracking")

        # Load the default puzzle
        self.load_selected_puzzle()

    def load_selected_puzzle(self, event = None):
        difficulty = self.difficulty_var.get()
        puzzle_number = self.puzzle_var.get()
        if difficulty == "Easy":
            self.puzzle = self.EASY_PUZZLES[puzzle_number - 1]
        elif difficulty == "Medium":
            self.puzzle = self.MEDIUM_PUZZLES[puzzle_number - 1]
        elif difficulty == "Hard":
            self.puzzle = self.HARD_PUZZLES[puzzle_number - 1]

        self.load_puzzle()

    def load_puzzle(self):
        for i in range(SIZE):
            for j in range(SIZE):
                value = self.puzzle[i][j]
                if value != 0:
                    self.board[i][j].set(value)
                else:
                    self.board[i][j].set("")

    def solve(self):
        self.load_selected_puzzle()  # Ensure the puzzle is loaded before solving
        solver = SudokuSolver(self.puzzle, self.algorithm_var.get())
        self.puzzle = solver.solve_with_algorithm()
        self.load_puzzle()



    def reset(self):
        self.load_selected_puzzle()


EASY_PUZZLES = [
        # EASY Puzzle 1
        [[5, 3, 0, 0, 7, 0, 0, 0, 0],
         [6, 0, 0, 1, 9, 5, 0, 0, 0],
         [0, 9, 8, 0, 0, 0, 0, 6, 0],
         [8, 0, 0, 0, 6, 0, 0, 0, 3],
         [4, 0, 0, 8, 0, 3, 0, 0, 1],
         [7, 0, 0, 0, 2, 0, 0, 0, 6],
         [0, 6, 0, 0, 0, 0, 2, 8, 0],
         [0, 0, 0, 4, 1, 9, 0, 0, 5],
         [0, 0, 0, 0, 8, 0, 0, 7, 9]],
        # EASY Puzzle 2
         [[0, 0, 4, 8, 0, 0, 0, 1, 7],
         [6, 7, 0, 9, 0, 0, 0, 0, 0],
         [5, 0, 8, 0, 3, 0, 0, 0, 4],
         [3, 0, 0, 7, 4, 0, 1, 0, 0],
         [0, 6, 9, 0, 0, 0, 7, 8, 0],
         [0, 0, 1, 0, 6, 9, 0, 0, 5],
         [1, 0, 0, 0, 8, 0, 3, 0, 6],
         [0, 0, 0, 0, 0, 6, 0, 9, 1],
         [2, 4, 0, 0, 0, 1, 5, 0, 0]],
        # EASY Puzzle 3
         [[8, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 3, 6, 0, 0, 0, 0, 0],
         [0, 7, 0, 0, 9, 0, 2, 0, 0],
         [0, 5, 0, 0, 0, 7, 0, 0, 0],
         [0, 0, 0, 0, 4, 5, 7, 0, 0],
         [0, 0, 0, 1, 0, 0, 0, 3, 0],
         [0, 0, 1, 0, 0, 0, 0, 6, 8],
         [0, 0, 8, 5, 0, 0, 0, 1, 0],
         [0, 9, 0, 0, 0, 0, 4, 0, 0]],
        # EASY Puzzle 4
         [[5, 3, 0, 0, 7, 0, 0, 0, 0],
         [6, 0, 0, 1, 9, 5, 0, 0, 0],
         [0, 9, 8, 0, 0, 0, 0, 6, 0],
         [8, 0, 0, 0, 6, 0, 0, 0, 3],
         [4, 0, 0, 8, 0, 3, 0, 0, 1],
         [7, 0, 0, 0, 2, 0, 0, 0, 6],
         [0, 6, 0, 0, 0, 0, 2, 8, 0],
         [0, 0, 0, 4, 1, 9, 0, 0, 5],
         [0, 0, 0, 0, 8, 0, 0, 7, 9]]
    ]

MEDIUM_PUZZLES = [
    # Medium Puzzle 1
    [
        [0, 0, 0, 2, 0, 5, 0, 0, 0],
        [0, 9, 0, 0, 0, 0, 7, 3, 0],
        [0, 0, 2, 0, 0, 9, 0, 6, 0],
        [2, 0, 0, 0, 0, 0, 4, 0, 9],
        [0, 0, 0, 0, 7, 0, 0, 0, 0],
        [6, 0, 9, 0, 0, 0, 0, 0, 1],
        [0, 8, 0, 4, 0, 0, 1, 0, 0],
        [0, 6, 3, 0, 0, 0, 0, 8, 0],
        [0, 0, 0, 6, 0, 8, 0, 0, 0]
    ],
    # Medium Puzzle 2
    [
        [0, 2, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 6, 0, 0, 0, 0, 3],
        [0, 7, 4, 0, 8, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 3, 0, 0, 2],
        [0, 8, 0, 0, 4, 0, 0, 1, 0],
        [6, 0, 0, 5, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 7, 8, 0],
        [5, 0, 0, 0, 0, 9, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 4, 0]
    ],
    # Medium Puzzle 3
    [
        [4, 0, 0, 0, 0, 6, 9, 0, 2],
        [0, 0, 0, 0, 0, 0, 0, 0, 3],
        [0, 0, 0, 0, 8, 7, 4, 6, 1],
        [0, 0, 5, 0, 0, 0, 0, 9, 0],
        [3, 0, 2, 0, 0, 0, 5, 0, 8],
        [0, 4, 0, 0, 0, 0, 1, 0, 0],
        [1, 7, 8, 6, 9, 0, 0, 0, 0],
        [5, 0, 0, 0, 0, 0, 0, 0, 0],
        [2, 0, 9, 8, 0, 0, 0, 0, 6]
    ],
    # Medium Puzzle 4
    [
        [0, 0, 0, 0, 0, 7, 0, 0, 0],
        [0, 0, 0, 0, 4, 5, 7, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 2],
        [0, 9, 6, 0, 0, 0, 0, 0, 0],
        [0, 8, 7, 0, 0, 0, 0, 6, 0],
        [0, 0, 0, 0, 0, 0, 9, 8, 0],
        [0, 5, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 3, 2, 0, 0, 0, 5, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0]
    ]
]

HARD_PUZZLES = [
    # Hard Puzzle 1
    [
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 3, 0, 8, 5],
        [0, 0, 1, 0, 2, 0, 0, 0, 0],
        [0, 0, 0, 5, 0, 7, 0, 0, 0],
        [0, 0, 4, 0, 0, 0, 1, 0, 0],
        [0, 9, 0, 0, 0, 0, 0, 0, 0],
        [5, 0, 0, 0, 0, 0, 0, 7, 3],
        [0, 0, 2, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 4, 0, 0, 0, 9]
    ],
    # Hard Puzzle 2
    [
        [0, 2, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 6, 0, 0, 0, 0, 3],
        [0, 7, 4, 0, 8, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 3, 0, 0, 2],
        [0, 8, 0, 0, 4, 0, 0, 1, 0],
        [6, 0, 0, 5, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 7, 8, 0],
        [5, 0, 0, 0, 0, 9, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 4, 0]
    ],
    # Hard Puzzle 3
    [
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 3, 0, 8, 5],
        [0, 0, 1, 0, 2, 0, 0, 0, 0],
        [0, 0, 0, 5, 0, 7, 0, 0, 0],
        [0, 0, 4, 0, 0, 0, 1, 0, 0],
        [0, 9, 0, 0, 0, 0, 0, 0, 0],
        [5, 0, 0, 0, 0, 0, 0, 7, 3],
        [0, 0, 2, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 4, 0, 0, 0, 9]
    ],
    # Hard Puzzle 4
    [
        [0, 2, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 6, 0, 0, 0, 0, 3],
        [0, 7, 4, 0, 8, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 3, 0, 0, 2],
        [0, 8, 0, 0, 4, 0, 0, 1, 0],
        [6, 0, 0, 5, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 7, 8, 0],
        [5, 0, 0, 0, 0, 9, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 4, 0]
    ]
]



root = tk.Tk()
app = SudokuGUI(root, EASY_PUZZLES, MEDIUM_PUZZLES, HARD_PUZZLES)
root.mainloop()



True
[[5, 3, 4, 6, 7, 8, 9, 1, 2], [6, 7, 2, 1, 9, 5, 3, 4, 8], [1, 9, 8, 3, 4, 2, 5, 6, 7], [8, 5, 9, 7, 6, 1, 4, 2, 3], [4, 2, 6, 8, 5, 3, 7, 9, 1], [7, 1, 3, 9, 2, 4, 8, 5, 6], [9, 6, 1, 5, 3, 7, 2, 8, 4], [2, 8, 7, 4, 1, 9, 6, 3, 5], [3, 4, 5, 2, 8, 6, 1, 7, 9]]
[[5, 3, 4, 6, 7, 8, 9, 1, 2], [6, 7, 2, 1, 9, 5, 3, 4, 8], [1, 9, 8, 3, 4, 2, 5, 6, 7], [8, 5, 9, 7, 6, 1, 4, 2, 3], [4, 2, 6, 8, 5, 3, 7, 9, 1], [7, 1, 3, 9, 2, 4, 8, 5, 6], [9, 6, 1, 5, 3, 7, 2, 8, 4], [2, 8, 7, 4, 1, 9, 6, 3, 5], [3, 4, 5, 2, 8, 6, 1, 7, 9]]


In [7]:
SIZE = 9  # Size of the Sudoku grid
BOX_SIZE = 3 #Size of Sudoku box

class SudokuCSP:
    def __init__(self, puzzle):
        self.variables = [(i, j) for i in range(SIZE) for j in range(SIZE)]
        self.values = puzzle
        self.domains = self.initialize_domains()
        self.neighbors = self.initialize_neighbors()

    def initialize_domains(self):
        domains = {}
        for i, j in self.variables:
            if self.values[i][j] == 0:
                domains[(i, j)] = set(range(1, 10))
            else:
                domains[(i, j)] = {self.values[i][j]}
        return domains

    def initialize_neighbors(self):
        neighbors = defaultdict(set)
        for i, j in self.variables:
            for x in range(SIZE):
                neighbors[(i, j)].add((i, x))
                neighbors[(i, j)].add((x, j))
            start_row, start_col = i - i % BOX_SIZE, j - j % BOX_SIZE
            for x, y in product(range(BOX_SIZE), repeat=2):
                neighbors[(i, j)].add((x + start_row, y + start_col))
            neighbors[(i, j)].remove((i, j))
        return neighbors

    def arcs(self):
        return [(var, neighbor) for var in self.variables for neighbor in self.neighbors[var]]

    def constraints(self, Xi, x, Xj, y):
        return x != y

    def revise(self, Xi, Xj):
        revised = False
        for x in list(self.domains[Xi]):
            if not any(self.constraints(Xi, x, Xj, y) for y in self.domains[Xj]):
                self.domains[Xi].remove(x)
                revised = True
        return revised

    def ac3(self):
        queue = deque(self.arcs())
        while queue:
            Xi, Xj = queue.popleft()
            if self.revise(Xi, Xj):
                if not self.domains[Xi]:
                    return False
                for Xk in self.neighbors[Xi]:
                    if Xk != Xj:
                        queue.append((Xk, Xi))
        return True

    def is_complete(self):
        return all(len(self.domains[var]) == 1 for var in self.variables)

    def solution(self):
        if not self.ac3():
            print("AC-3 algorithm failed to find a solution.")
            return False
        solved_puzzle = [[self.domains[(i, j)].pop() for j in range(SIZE)] for i in range(SIZE)]
        return solved_puzzle


class SudokuSolver:
    def __init__(self, puzzle, algo):
        self.algorithm = algo
        self.board = [[0] * SIZE for _ in range(SIZE)]
        if puzzle:
            self.set_puzzle(puzzle)

    def set_puzzle(self, puzzle):
        for i, row in enumerate(puzzle):
            for j, val in enumerate(row):
                self.board[i][j] = val

    def is_valid_move(self, row, col, num):
        for i in range(SIZE):
            if self.board[row][i] == num or self.board[i][col] == num:
                return False
        start_row, start_col = row - row % BOX_SIZE, col - col % BOX_SIZE
        for i, j in product(range(BOX_SIZE), repeat=2):
            if self.board[i + start_row][j + start_col] == num:
                return False
        return True

    def find_empty(self):
        for i in range(SIZE):
            for j in range(SIZE):
                if self.board[i][j] == 0:
                    return i, j
        return None

    def solve_backtracking(self):
        empty = self.find_empty()
        if not empty:
            return True
        row, col = empty
        for num in range(1, SIZE + 1):
            if self.is_valid_move(row, col, num):
                self.board[row][col] = num
                if self.solve_backtracking():
                    return True
                self.board[row][col] = 0
        return False

    def solve_ac3(self):
        csp = SudokuCSP(self.board)
        result = csp.solution()
        if result:
            self.set_puzzle(csp.values)
        return result
    
    def solve_with_algorithm(self):
        if self.algorithm == 'arc':
            solved_puzzle = self.solve_ac3()
        elif self.algorithm == 'backtracking':
            print(self.solve_backtracking())
            solved_puzzle = self.board
        print(solved_puzzle)  # Assign the solved puzzle to solved_puzzle
        return solved_puzzle  # Return the solved puzzle


class SudokuGUI:
    def __init__(self, root, easy_puzzles, medium_puzzles, hard_puzzles):
        self.root = root
        self.EASY_PUZZLES = easy_puzzles
        self.MEDIUM_PUZZLES = medium_puzzles
        self.HARD_PUZZLES = hard_puzzles
        self.puzzle = [[]]
        self.root.title("Welcome to Sudoko Solver!")

        # Create frame for Sudoku grid
        grid_frame = ttk.Frame(self.root)
        grid_frame.grid(row=0, column=0, padx=10, pady=10)

        self.board = [[tk.StringVar() for _ in range(SIZE)] for _ in range(SIZE)]
        self.labels = []
        for i in range(SIZE):
            for j in range(SIZE):
                label = tk.Label(grid_frame, width=3, highlightthickness=1, highlightbackground='#FF5733', textvariable=self.board[i][j])    
                label.grid(row=i, column=j, ipadx=5, ipady=5)
                label.bind('<KeyPress>', lambda event, row=i, col=j: self.validate_input(event, row, col))  # Bind event to validate input
                self.labels.append(label)

        # Create a frame for buttons and dropdown menu
        control_frame = ttk.Frame(self.root)
        control_frame.grid(row=0, column=1, padx=10, pady=10)

        # Create dropdown menu for selecting difficulty level
        difficulty_label = ttk.Label(control_frame, text="Difficulty Level:", font=('TkDefaultFont', 10, 'bold'))
        difficulty_label.grid(row=0, column=0, columnspan=2, padx=10, pady=5)
        self.difficulty_var = tk.StringVar()
        difficulty_combobox = ttk.Combobox(control_frame, textvariable=self.difficulty_var, values=["Easy", "Medium", "Hard"], state="readonly", width=10)
        difficulty_combobox.grid(row=1, column=0, columnspan=2, padx=10, pady=5)
        difficulty_combobox.current(0)  # Set default selection to "Easy"
        difficulty_combobox.bind("<<ComboboxSelected>>", self.load_selected_puzzle)


        # Create Solve button
        solve_button = tk.Button(control_frame, text="Solve", command=self.solve, width=10)
        solve_button.grid(row=2, column=0, padx=10, pady=5)

        # Create Reset button
        reset_button = tk.Button(control_frame, text="Reset", command=self.reset, width=10)
        reset_button.grid(row=2, column=1, padx=10, pady=5)

        # Create radio buttons for choosing puzzle
        puzzle_label = ttk.Label(control_frame, text="Choose Puzzle:", font=('TkDefaultFont', 10, 'bold'))
        puzzle_label.grid(row=3, column=0, columnspan=2, padx=10, pady=5)
        self.puzzle_var = tk.IntVar()
        for i in range(1, 5):
            puzzle_radio = ttk.Radiobutton(control_frame, text=f"Puzzle {i}", variable=self.puzzle_var, value=i, command=self.load_selected_puzzle)
            puzzle_radio.grid(row=4, column=i-1, padx=5, pady=5)

        self.puzzle_var.set(1)

        # Create radio buttons for selecting algorithm
        algorithm_label = ttk.Label(control_frame, text="Choose Algorithm:", font=('TkDefaultFont', 10, 'bold'))
        algorithm_label.grid(row=5, column=0, columnspan=2, padx=10, pady=5)
        self.algorithm_var = tk.StringVar()
        backtracking_radio = ttk.Radiobutton(control_frame, text="Backtracking", variable=self.algorithm_var, value="backtracking")
        backtracking_radio.grid(row=6, column=0, padx=5, pady=5)

        ac3_radio = ttk.Radiobutton(control_frame, text="AC-3", variable=self.algorithm_var, value="arc")
        ac3_radio.grid(row=6, column=1, padx=5, pady=5)
        self.algorithm_var.set("backtracking")

        # Load the default puzzle
        self.load_selected_puzzle()

    def load_selected_puzzle(self, event = None):
        difficulty = self.difficulty_var.get()
        puzzle_number = self.puzzle_var.get()
        if difficulty == "Easy":
            self.puzzle = self.EASY_PUZZLES[puzzle_number - 1]
        elif difficulty == "Medium":
            self.puzzle = self.MEDIUM_PUZZLES[puzzle_number - 1]
        elif difficulty == "Hard":
            self.puzzle = self.HARD_PUZZLES[puzzle_number - 1]

        self.load_puzzle()

    def load_puzzle(self):
        for i in range(SIZE):
            for j in range(SIZE):
                value = self.puzzle[i][j]
                if value != 0:
                    self.board[i][j].set(value)
                else:
                    self.board[i][j].set("")

    def solve(self):
        self.load_selected_puzzle()  # Ensure the puzzle is loaded before solving
        solver = SudokuSolver(self.puzzle, self.algorithm_var.get())
        self.puzzle = solver.solve_with_algorithm()
        self.load_puzzle()



    def reset(self):
        self.load_selected_puzzle()


EASY_PUZZLES = [
        # EASY Puzzle 1
        [[5, 3, 0, 0, 7, 0, 0, 0, 0],
         [6, 0, 0, 1, 9, 5, 0, 0, 0],
         [0, 9, 8, 0, 0, 0, 0, 6, 0],
         [8, 0, 0, 0, 6, 0, 0, 0, 3],
         [4, 0, 0, 8, 0, 3, 0, 0, 1],
         [7, 0, 0, 0, 2, 0, 0, 0, 6],
         [0, 6, 0, 0, 0, 0, 2, 8, 0],
         [0, 0, 0, 4, 1, 9, 0, 0, 5],
         [0, 0, 0, 0, 8, 0, 0, 7, 9]],
        # EASY Puzzle 2
         [[0, 0, 4, 8, 0, 0, 0, 1, 7],
         [6, 7, 0, 9, 0, 0, 0, 0, 0],
         [5, 0, 8, 0, 3, 0, 0, 0, 4],
         [3, 0, 0, 7, 4, 0, 1, 0, 0],
         [0, 6, 9, 0, 0, 0, 7, 8, 0],
         [0, 0, 1, 0, 6, 9, 0, 0, 5],
         [1, 0, 0, 0, 8, 0, 3, 0, 6],
         [0, 0, 0, 0, 0, 6, 0, 9, 1],
         [2, 4, 0, 0, 0, 1, 5, 0, 0]],
        # EASY Puzzle 3
         [[8, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 3, 6, 0, 0, 0, 0, 0],
         [0, 7, 0, 0, 9, 0, 2, 0, 0],
         [0, 5, 0, 0, 0, 7, 0, 0, 0],
         [0, 0, 0, 0, 4, 5, 7, 0, 0],
         [0, 0, 0, 1, 0, 0, 0, 3, 0],
         [0, 0, 1, 0, 0, 0, 0, 6, 8],
         [0, 0, 8, 5, 0, 0, 0, 1, 0],
         [0, 9, 0, 0, 0, 0, 4, 0, 0]],
        # EASY Puzzle 4
         [[5, 3, 0, 0, 7, 0, 0, 0, 0],
         [6, 0, 0, 1, 9, 5, 0, 0, 0],
         [0, 9, 8, 0, 0, 0, 0, 6, 0],
         [8, 0, 0, 0, 6, 0, 0, 0, 3],
         [4, 0, 0, 8, 0, 3, 0, 0, 1],
         [7, 0, 0, 0, 2, 0, 0, 0, 6],
         [0, 6, 0, 0, 0, 0, 2, 8, 0],
         [0, 0, 0, 4, 1, 9, 0, 0, 5],
         [0, 0, 0, 0, 8, 0, 0, 7, 9]]
    ]

MEDIUM_PUZZLES = [
    # Medium Puzzle 1
    [
        [0, 0, 0, 2, 0, 5, 0, 0, 0],
        [0, 9, 0, 0, 0, 0, 7, 3, 0],
        [0, 0, 2, 0, 0, 9, 0, 6, 0],
        [2, 0, 0, 0, 0, 0, 4, 0, 9],
        [0, 0, 0, 0, 7, 0, 0, 0, 0],
        [6, 0, 9, 0, 0, 0, 0, 0, 1],
        [0, 8, 0, 4, 0, 0, 1, 0, 0],
        [0, 6, 3, 0, 0, 0, 0, 8, 0],
        [0, 0, 0, 6, 0, 8, 0, 0, 0]
    ],
    # Medium Puzzle 2
    [
        [0, 2, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 6, 0, 0, 0, 0, 3],
        [0, 7, 4, 0, 8, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 3, 0, 0, 2],
        [0, 8, 0, 0, 4, 0, 0, 1, 0],
        [6, 0, 0, 5, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 7, 8, 0],
        [5, 0, 0, 0, 0, 9, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 4, 0]
    ],
    # Medium Puzzle 3
    [
        [4, 0, 0, 0, 0, 6, 9, 0, 2],
        [0, 0, 0, 0, 0, 0, 0, 0, 3],
        [0, 0, 0, 0, 8, 7, 4, 6, 1],
        [0, 0, 5, 0, 0, 0, 0, 9, 0],
        [3, 0, 2, 0, 0, 0, 5, 0, 8],
        [0, 4, 0, 0, 0, 0, 1, 0, 0],
        [1, 7, 8, 6, 9, 0, 0, 0, 0],
        [5, 0, 0, 0, 0, 0, 0, 0, 0],
        [2, 0, 9, 8, 0, 0, 0, 0, 6]
    ],
    # Medium Puzzle 4
    [
        [0, 0, 0, 0, 0, 7, 0, 0, 0],
        [0, 0, 0, 0, 4, 5, 7, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 2],
        [0, 9, 6, 0, 0, 0, 0, 0, 0],
        [0, 8, 7, 0, 0, 0, 0, 6, 0],
        [0, 0, 0, 0, 0, 0, 9, 8, 0],
        [0, 5, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 3, 2, 0, 0, 0, 5, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0]
    ]
]

HARD_PUZZLES = [
    # Hard Puzzle 1
    [
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 3, 0, 8, 5],
        [0, 0, 1, 0, 2, 0, 0, 0, 0],
        [0, 0, 0, 5, 0, 7, 0, 0, 0],
        [0, 0, 4, 0, 0, 0, 1, 0, 0],
        [0, 9, 0, 0, 0, 0, 0, 0, 0],
        [5, 0, 0, 0, 0, 0, 0, 7, 3],
        [0, 0, 2, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 4, 0, 0, 0, 9]
    ],
    # Hard Puzzle 2
    [
        [0, 2, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 6, 0, 0, 0, 0, 3],
        [0, 7, 4, 0, 8, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 3, 0, 0, 2],
        [0, 8, 0, 0, 4, 0, 0, 1, 0],
        [6, 0, 0, 5, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 7, 8, 0],
        [5, 0, 0, 0, 0, 9, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 4, 0]
    ],
    # Hard Puzzle 3
    [
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 3, 0, 8, 5],
        [0, 0, 1, 0, 2, 0, 0, 0, 0],
        [0, 0, 0, 5, 0, 7, 0, 0, 0],
        [0, 0, 4, 0, 0, 0, 1, 0, 0],
        [0, 9, 0, 0, 0, 0, 0, 0, 0],
        [5, 0, 0, 0, 0, 0, 0, 7, 3],
        [0, 0, 2, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 4, 0, 0, 0, 9]
    ],
    # Hard Puzzle 4
    [
        [0, 2, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 6, 0, 0, 0, 0, 3],
        [0, 7, 4, 0, 8, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 3, 0, 0, 2],
        [0, 8, 0, 0, 4, 0, 0, 1, 0],
        [6, 0, 0, 5, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 7, 8, 0],
        [5, 0, 0, 0, 0, 9, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 4, 0]
    ]
]



root = tk.Tk()
app = SudokuGUI(root, EASY_PUZZLES, MEDIUM_PUZZLES, HARD_PUZZLES)
root.mainloop()



AC-3 algorithm failed to find a solution.
False


Exception in Tkinter callback
Traceback (most recent call last):
  File "E:\ProgramData\anaconda3\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\M O I Z\AppData\Local\Temp\ipykernel_18336\3595824645.py", line 221, in solve
    self.load_puzzle()
  File "C:\Users\M O I Z\AppData\Local\Temp\ipykernel_18336\3595824645.py", line 211, in load_puzzle
    value = self.puzzle[i][j]
            ~~~~~~~~~~~^^^
TypeError: 'bool' object is not subscriptable
